In [22]:
import storage as storage
import os
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="XXX",
  database="forestry"
)

mycursor = mydb.cursor()

def inDatabase(filename):
    mycursor.execute("SELECT filename FROM PolicyPapers WHERE filename='%s'" % filename)
    return mycursor.fetchone()

def getCountries(top_three):
    country1 = None
    country2 = None
    country3 = None
    if len(top_three) == 1:
        country1 = top_three[0]
    if len(top_three) == 2:
        country2 = top_three[1]
    elif len(top_three) == 3:
        country3 = top_three[2]
        
    return country1, country2, country3

def getFirst(lst):
    result = None
    if len(lst) > 0:
        result = lst[0]
    return result

def insertPaper(paper):
    sql_insert = "INSERT INTO PolicyPapers " \
        "(filename, country1, country2, country3, " \
        "date, title, extractedText, cleanedText)" \
        "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

    country1, country2, country3 = getCountries(paper.country[:3])

    val = (paper.filename, country1, country2, country3, getFirst(paper.date),
        getFirst(paper.title), ' '.join(paper.extractedText), None)
    mycursor.execute(sql_insert, val)

    mydb.commit()

    print(mycursor.rowcount, "record inserted.")

In [23]:
def extractAndSavePolicyPaper(file_path):
    paper = storage.PolicyPaper(file_path)
    insertPaper(paper)

# Might need to generalize multiple levels of directories
def extractPolicyPapersFromDirectory(directory_path):    
    # Retrieve file names inside folder
    files = os.listdir(directory_path)
    print("Folder now open.")
 
    for file in files:
        if file == ".DS_Store":
            continue

        file_path = directory_path + "/" + file
        if not inDatabase(storage.getFileName(file_path)):
            extractAndSavePolicyPaper(file_path)
        else:
            print("PROCESSING", file_path)

In [24]:
extractPolicyPapersFromDirectory("sample data/Kenya")

Folder now open.
PROCESSING sample data/Kenya/Kefristrategicplan2013-2018.pdf
PROCESSING sample data/Kenya/Sessional-paper-on-Kenya-National-Land-Policy.pdf
PROCESSING sample data/Kenya/Proposal_NFRA.pdf
Paper successfully converted!
On page 0: 
['The', 'Agriculture,', 'Fisheries', 'and', 'Food', 'Authority', 'Act,', '2013'] True False <re.Match object; span=(0, 4), match='2013'>
['The', 'common', 'seal'] True False None
['The', 'respective', 'secretariats', 'or', 'management'] True False None
['The', 'Board', 'shall', 'cause', 'minutes', 'of', 'all', 'resolutions'] True False None
Metadata extracted!
DONE
1 record inserted.
Paper successfully converted!
On page 0: 
['This', 'Act', 'may', 'be', 'cited', 'as', 'the', 'Land', 'Act,', '2012.'] False True <re.Match object; span=(0, 4), match='2012'>
['The', 'Cabinet', 'Secretary', 'shall,', 'in', 'relation', 'to', 'the', 'management', 'and', 'administration'] True False None
['The', 'Commission', 'may', 'declare', 'a', 'licence', 'granted'

1 record inserted.
Paper successfully converted!
On page 0: 
['This', 'Act', 'may', 'be', 'cited', 'as', 'the', 'Climate', 'Change', 'Act,', '2016.'] False True <re.Match object; span=(0, 4), match='2016'>
['The', 'Council', 'shall', 'provide', 'an', 'overarching', 'national', 'climate', 'change', 'coordination'] True False None
['The', 'Council', 'shall,', 'by', 'Regulations,', 'prescribe', 'a', 'Code', 'of', 'Conduct', 'for', 'members'] True False None
['The', 'Council,', 'on', 'the', 'recommendation', 'of', 'the', 'Cabinet', 'Secretary', 'shall', 'each', 'year'] True False None
['The', 'Authority', 'shall', 'integrate', 'climate', 'risk', 'and', 'vulnerability', 'assessment', 'into', 'all'] True False None
['The', 'Cabinet', 'Secretary', 'shall', 'make', 'regulations', 'to', 'guide', 'the', 'reporting', 'and'] True False None
['The', 'financial', 'year', 'of', 'the', 'Council', 'shall', 'be', 'the', 'period', 'of', 'twelve', 'months', 'ending'] True False None
['The', 'members', 'of